In [7]:
# import libraries
import pandas as pd
import sqlite3 as db
import bcrypt

# create a connection to the database
# conn = db.connect('Data/data.db')
conn = db.connect('data.db')
# create a cursor object
cur = conn.cursor()


# Create Stock Data Table

In [ ]:

conn.execute('''
    CREATE TABLE stock_data (
    StockEntryID INTEGER,
    StockID INTEGER,
    StockSymbol TEXT,
    Timestamp TEXT,
    TimeFrame INTEGER,
    OpenPrice REAL,
    ClosePrice REAL,
    HighPrice REAL,
    LowPrice REAL,
    Volume REAL,
    PRIMARY KEY (StockEntryID, StockID , TimeFrame)  
)
''')


In [75]:
# create new employee table
conn.execute('''
    CREATE TABLE employee (
    EmployeeID INTEGER PRIMARY KEY,
    FirstName TEXT NOT NULL,
    LastName TEXT NOT NULL,
    Department TEXT NOT NULL,
    Salary REAL NOT NULL
)
''')


# Create Patterns Table

In [ ]:

conn.execute('''
    CREATE TABLE IF NOT EXISTS patterns (
        PatternID INTEGER ,
        StockID INTEGER,
        ClusterID INTEGER,
        PricePoints TEXT,
        VolumeData TEXT,
        SentimentScore REAL,
        TimeSpan VARCHAR(50),
        MarketCondition VARCHAR(20),
        Outcome REAL,
        Label VARCHAR(50),
        PRIMARY KEY (PatternID, StockID) 
    )
''')

# create cluster Table


In [ ]:


conn.execute('''
    CREATE TABLE IF NOT EXISTS clusters (
        ClusterID INTEGER ,
        ClusterName VARCHAR(50),
        StockID INTEGER,
        Description TEXT,
        AVGPricePoints TEXT,
        AVGPercentageChanges TEXT,
        AVGVolumeData TEXT,
        SentimentScore REAL,
        MarketCondition VARCHAR(20),
        Outcome REAL,
        Label VARCHAR(50),
        ProbabilityScore REAL,
        Pattern_Count INTEGER,
        PRIMARY KEY (ClusterID, StockID) 
    )
''')



# Create User Table

In [ ]:


conn.execute('''
    CREATE TABLE IF NOT EXISTS users (
        UserID INTEGER PRIMARY KEY,
        Username VARCHAR(50) NOT NULL,
        Password VARCHAR(255) NOT NULL,
        Email VARCHAR(100) NOT NULL,
        Preferences TEXT
    )
''')


# Create Prediction Table


In [ ]:

conn.execute('''
    CREATE TABLE IF NOT EXISTS predictions (
        PredictionID INTEGER PRIMARY KEY AUTOINCREMENT,
        StockID INTEGER,
        PatternID INTEGER,
        SentimentID INTEGER,
        PredictionDate DATETIME DATETIME DEFAULT CURRENT_TIMESTAMP,
        PredictedOutcome TEXT CHECK(json_valid(PredictedOutcome)),
        ConfidenceLevel FLOAT
    )
''')
conn.commit()

# Create Notification Table

In [10]:
# Notification Table
# Column Name	Data Type	Description
# NotificationID	INT (PK)	Unique identifier for notification
# UserID	INT (FK)	Foreign key referencing User
# PredictionID	INT (FK)	Foreign key referencing Prediction
# SentTime	DATETIME	Time notification was sent
# NotificationType	VARCHAR(50)	Type of notification
# Status	VARCHAR(20)	Notification statusc

conn.execute('''
    CREATE TABLE IF NOT EXISTS notifications (
        NotificationID INTEGER PRIMARY KEY AUTOINCREMENT,
        UserID INTEGER,
        PredictionID INTEGER,
        SentTime DATETIME DEFAULT CURRENT_TIMESTAMP,
        NotificationType VARCHAR(50),
        Status VARCHAR(20)
    )
''')
conn.commit()

# Add ( Insert )

In [ ]:
password = "1234"
# hash the password
hashed_password = bcrypt.hashpw(password.encode('utf-8'), bcrypt.gensalt()) 
# insert new admin user
conn.execute('''
    INSERT INTO users (Username, Password, Email, Preferences) values (?, ?, ?, ?)
    ''' , ("admin",hashed_password,"admin@admin.com","admin"))
conn.commit()

             

# Alter

In [19]:
# # alter table patterns to add new columns , MaxGain real , MaxDrawdown real
# cur.execute('''
#     ALTER TABLE patterns ADD COLUMN MaxGain REAL
# ''')
# cur.execute('''
#     ALTER TABLE patterns ADD COLUMN MaxDrawdown REAL
# ''')
# # alter table clusters to add new columns , MaxGain real , MaxDrawdown real
# cur.execute('''
#     ALTER TABLE clusters ADD COLUMN MaxGain REAL
# ''')
# cur.execute('''
#     ALTER TABLE clusters ADD COLUMN MaxDrawdown REAL
# ''')
# add column TimeFrame to stock_data table
# cur.execute('''
#     ALTER TABLE stock_data ADD COLUMN TimeFrame INTEGER
# ''')

# alter table stock_data to male the combined primary key of StockID and stockEntryID and the timeframe

cur.execute('''
    ALTER TABLE stock_data DROP PRIMARY KEY
''')
cur.execute('''
    ALTER TABLE stock_data ADD PRIMARY KEY (StockID, StockEntryID, TimeFrame)
''')
            

OperationalError: near "PRIMARY": syntax error

# select

In [78]:
# select stock data where StockSymbol = 'XAUUSD' and OpenPrice < 1274.7
df = pd.read_sql_query("SELECT * FROM stock_data where StockSymbol='XAUUSD' and OpenPrice < 1274.7", conn)
# print the dataframe
print(df)

       StockEntryID  StockID StockSymbol            Timestamp  OpenPrice  \
0                 1        1      XAUUSD  2014-02-10 12:00:00    1273.39   
1                 2        1      XAUUSD  2014-02-10 13:00:00    1272.99   
2                 3        1      XAUUSD  2014-02-10 14:00:00    1272.52   
3                 4        1      XAUUSD  2014-02-10 15:00:00    1274.44   
4                 5        1      XAUUSD  2014-02-10 16:00:00    1274.22   
...             ...      ...         ...                  ...        ...   
18682         30995        1      XAUUSD  2019-05-23 07:00:00    1273.02   
18683         30996        1      XAUUSD  2019-05-23 08:00:00    1273.07   
18684         30997        1      XAUUSD  2019-05-23 09:00:00    1274.41   
18685         30998        1      XAUUSD  2019-05-23 10:00:00    1273.93   
18686         30999        1      XAUUSD  2019-05-23 11:00:00    1274.50   

       ClosePrice  HighPrice  LowPrice  Volume  
0         1272.99    1273.88   1272.43

# Delete

In [45]:
# remove all entries from the stock_data table
# conn.execute('DELETE FROM stock_data')
conn.execute('DELETE FROM clusters')
conn.execute('DELETE FROM patterns')
conn.commit()

# Drop

In [8]:
# drop notifications table
conn.execute('DROP TABLE IF EXISTS notifications')

conn.commit()

# Close Connection

In [6]:
conn.close()